## 🤗 Transfer learning и Huggingface

Благодаря тому, что большие сети обладают хорошей обобщающей способностью, мы можем брать предобученные сети и использовать их в своих целях. Кроме того, сети можно дообучать на своих задачах.

Очень пополулярной библиотекой, хостингом для хранения и инференса (запуска) моделей является 🤗 **[Huggingface](https://huggingface.co/)**.



## mBART huggingface
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210
* Model: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 10.0 MB/s 
     |████████████████████████████████| 1.2 MB 66.3 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#### Загружаем модели

In [2]:
from transformers import AutoTokenizer, MBartForConditionalGeneration #AutoModel

model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

In [3]:
print(type(tokenizer), type(model))

<class 'transformers.models.mbart.tokenization_mbart_fast.MBartTokenizerFast'> <class 'transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration'>


In [23]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [19]:
# по сути данные те же можно пользоваться любым способом загрузки
from datasets import load_dataset

dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")["test"]

No config specified, defaulting to: gazeta/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
!head -n 1 gazeta_train.txt
!cat gazeta_train.txt | wc -l
!cat gazeta_val.txt | wc -l
!cat gazeta_test.txt | wc -l

head: cannot open 'gazeta_train.txt' for reading: No such file or directory
cat: gazeta_train.txt: No such file or directory
0
cat: gazeta_val.txt: No such file or directory
0
cat: gazeta_test.txt: No such file or directory
0


In [21]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [24]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [46]:
test_records[0]["text"]

'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, как они и им подобные объекты образовались в космосе». Каждый проект из выбранн

#### Суммаризация текста

In [ ]:
article_text = """Колизей или амфитеатр Флавиев (лат. Amphitheatrum Flavium) — амфитеатр,
памятник архитектуры Древнего Рима, наиболее известное и одно из самых грандиозных сооружений Древнего мира, сохранившихся до нашего времени.
Находится в Риме, в низине между Эсквилинским, Палатинским и Целиевым холмами.
Строительство самого большого амфитеатра античного мира, вместимостью свыше 50 тыс. 
человек, велось на протяжении восьми лет как коллективное сооружение императоров династии Флавиев. 
Его начали строить в 72 году н. э. при императоре Веспасиане, а в 80 году н. э. амфитеатр был освящён императором Титом. 
Амфитеатр расположился на том месте, где был пруд, относившийся к Золотому дому Нерона.
История Колизея восходит к 68 году, когда измена преторианской гвардии и осуждение Сената 
заставили императора Нерона после четырнадцати лет деспотического управления государством покончить
с собой на загородной вилле неподалеку от Рима. Смерть Нерона привела к восемнадцатимесячной гражданской войне, 
закончившейся в 69 году. Победу в ней одержал Тит Флавий Веспасиан, которого в наши дни называют просто Веспасианом.
До того как стать императором, Веспасиан принимал участие в подавлении восстания иудеев, начавшегося в 66 году. 
После этого Веспасиан вместе с Титом в богатых восточных провинциях собирал налоги, чтобы привести в порядок 
государственные финансы, расстроенные Нероном и гражданской войной. Они возвратились в Рим в 71 году, чтобы отпраздновать победу над иудеями.
Став императором, Веспасиан решил реконструировать центр Рима и упрочить собственный культ, искоренив память о 
своём предшественнике Нероне. Оставалась нерешённой непростая проблема: что делать с дворцом Нерона, Золотым домом, 
как его называли, который вместе с прилегающим парком занимал в центре Рима площадь в 120 гектаров. 
Веспасиан решил разместить в нём имперские учреждения, а озеро возле дома засыпать и построить амфитеатр, 
предназначенный для развлечений народа. Это было хорошо взвешенное решение: постройкой амфитеатра земли, 
которыми пользовался Нерон, передавались народу.
"""


input_ids = tokenizer.prepare_seq2seq_batch(
    [article_text],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

output_ids = model.generate(
    input_ids=input_ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [5]:
print(summary)

В Древнем Риме был построен самый большой амфитеатр античного мира, вместимостью свыше 50 тыс. человек. Его строительство велось на протяжении восьми лет как коллективное сооружение императоров династии Флавиев. Победу в ней одержал Тит Флавий Веспасиан.


In [ ]:
print(model)

In [ ]:
!pip install razdel networkx pymorphy2[fast] nltk rouge==0.3.1 summa lexrank sumy
!pip install --upgrade datasets razdel spacy networkx seaborn scipy tqdm tensorflow-text
!python -m spacy download ru_core_news_md

In [9]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
test_records[0]

In [60]:
import razdel

def calc_lead_n_score(records, n=3, lower=True, nrows=10):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        input_ids = tokenizer.prepare_seq2seq_batch(
            [record['text']],
            src_lang="ru_XX", 
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=600
        )["input_ids"].cuda()

        output_ids = model.generate(
            input_ids=input_ids,
            max_length=162,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )[0]

        summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        references.append(summary)

        text = record['summary']
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)    

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Count: 10
Ref: Госдума одобрила в первом чтении внесенные президентом РФ Владимиром Путиным поправки в Конституцию. В его основу легли предложения, которые президент России Владимир Путин озвучил в послании Федеральному собранию. Одно из них — регулярная индексация, формирование пенсионной системы на основе принципов всеобщности, справедливости и солидарности поколений. Индексация пенсий работающим пенсионерам не производится с 2016 года. И менять это решение в правительстве не собирались до последнего времени.
Hyp: работающие пенсионеры дождались индексации. это следует из уже одобренных в первом чтении поправок в конституцию. так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. в россии их насчитывается более девяти миллионов. эксперты называют это восстановлением справедливости.
BLEU:  0.3094368388012075
ROUGE:  {'rouge-1': {'f': 0.1699699631986438, 'p': 0.20041374891757538, 'r': 0.15933133086367485}, 'rouge-2': {'f': 0.047887943381164005, 'p': 0